[![LibKet](../images/LibKet.png)](https://gitlab.com/mmoelle1/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/mmoelle1/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://mmoelle1.gitlab.io/LibKet/

---

# Tutorial \#2: Quantum computing today and future perspective

## Getting started

Let's include **LibKet**'s main headerfile and include its namespaces. This can take some time, stay tuned.

In [3]:
#include <array>
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

Let us create a simple quantum expression and evaluate it on different quantum backends. Don't forget to measure at the end since some backends do not support direct statevector readouts.

In [4]:
auto expr = measure(cnot(h(sel<0>()), sel<1>(init())));

## Cirq

[Cirq](https://quantumai.google/cirq) is a Python package for writing, manipulating, and optimizing quantum circuits developed by Google. It is seamlessly integrated into **LibKet** using the embedded Python interpreter, so you should not notice all the technical details going on under the hood.

As for the [QuEST](https://quest.qtechtheory.org) simulator, we simply create a quantum device for two qubits, load the quantum expression into it, and evaluate it.

In [13]:
QDevice<QDeviceType::cirq_simulator, 2> cirq;
utils::json result = cirq(expr).eval(1);

Unlike QuEST, Cirq returns the outcome of the evaluation as JSON object which we can print directly

In [4]:
std::cout << result << std::endl;

{"cirq_type":"Result","measurements":{"0":{"binary":true,"dtype":"int8","packed_digits":"00","shape":[1,1]},"1":{"binary":true,"dtype":"int8","packed_digits":"00","shape":[1,1]}},"params":{"cirq_type":"ParamResolver","param_dict":[]}}


We can also get a slightly prettier output

In [5]:
std::cout << result.dump(2) << std::endl;

{
  "cirq_type": "Result",
  "measurements": {
    "0": {
      "binary": true,
      "dtype": "int8",
      "packed_digits": "00",
      "shape": [
        1,
        1
      ]
    },
    "1": {
      "binary": true,
      "dtype": "int8",
      "packed_digits": "00",
      "shape": [
        1,
        1
      ]
    }
  },
  "params": {
    "cirq_type": "ParamResolver",
    "param_dict": []
  }
}


As this is still not too informative, **LibKet** provides quick access `get<...>()` functions to retrieve information about the ``duration`` of the quantum computation, a ``histogram`` of all measured states, and the ``best`` state, i.e. the one that was measured most often

In [6]:
std::cout << "duration  : " << cirq.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << cirq.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << cirq.get<QResultType::best>(result)             << std::endl;

duration  : 0 seconds
histogram : 1,0,0,0
best      : 0


Let's increase the number of **shots**, i.e. the number of times we run the quantum circuit in our simulator, to ``20`` to see how the histogram changes 

In [7]:
result = cirq.eval(20);

std::cout << "duration  : " << cirq.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << cirq.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << cirq.get<QResultType::best>(result)             << std::endl;

duration  : 0 seconds
histogram : 13,0,0,7
best      : 0


## Qiskit

Let's move on to the next quantum device supported by **LibKet** - [Qiskit](https://qiskit.org). Without any changes to the quantum expression we simply load it into a new Qiskit device and run it the same way as before

In [5]:
QDevice<QDeviceType::qiskit_qasm_simulator, 2> qiskit;
utils::json result = qiskit(expr).eval(20);

std::cout << "duration  : " << qiskit.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << qiskit.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << qiskit.get<QResultType::best>(result)             << std::endl;

duration  : 0.000701189 seconds
histogram : 11,0,0,9
best      : 0


Let's take a short look under the hood to see how much work is required to write low-level quantum assembly language (QASM) for the two simulators

In [12]:
std::cout << "--- Cirq ---\n" << cirq   << std::endl;
std::cout << "-- Qiskit --\n" << qiskit << std::endl;

--- Cirq ---
cirq.H.on(q[0])
cirq.CNOT.on(q[0],q[1])
cirq.measure(q[0])
cirq.measure(q[1])

-- Qiskit --
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0], q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];



**LibKet** is designed as backend-agnostic quantum programming framework with a unified API, which means that the core functionality is implemented for all quantum backends. If a particular backend does not support a specific function, e.g., Cirq does not report the ``duration`` of the quantum computation, default values are returned.

Some quantum backends provide extra functionality, which is exposed via **LibKet**. Cirq for instance can export the quantum circuit to [LaTeX](https://www.latex-project.org) code using the [Qcircuit](https://ctan.org/pkg/qcircuit) package

In [10]:
std::cout << cirq.to_latex() << std::endl;

\Qcircuit @R=1em @C=0.75em {
 \\
 &\lstick{\text{0}}& \qw&\gate{\text{H}} \qw&\control \qw    &\meter \qw&\qw\\
 &\lstick{\text{1}}& \qw&                \qw&\targ    \qw\qwx&\meter \qw&\qw\\
 \\
}


In [7]:
std::cout << qiskit.to_latex() << std::endl;

% \documentclass[preview]{standalone}
% If the image is too large to fit on this documentclass use
\documentclass[draft]{beamer}
% img_width = 3, img_depth = 6
\usepackage[size=custom,height=10,width=15,scale=0.7]{beamerposter}
% instead and customize the height and width (in cm) to fit.
% Large images may run out of memory quickly.
% To fix this use the LuaLaTeX compiler, which dynamically
% allocates memory.
\usepackage[braket, qm]{qcircuit}
\usepackage{amsmath}
\pdfmapfile{+sansmathaccent.map}
% \usepackage[landscape]{geometry}
% Comment out the above line if using the beamer documentclass.
\begin{document}

\begin{equation*}
    \Qcircuit @C=1.0em @R=0.0em @!R {
	 	\lstick{ {q}_{0} :  } & \gate{H} & \ctrl{1} & \meter & \qw & \qw & \qw\\
	 	\lstick{ {q}_{1} :  } & \qw & \targ & \qw & \meter & \qw & \qw\\
	 	\lstick{c:} & {/_{_{2}}} \cw & \cw & \dstick{0} \cw \cwx[-2] & \dstick{1} \cw \cwx[-1] & \cw & \cw\\
	 }
\end{equation*}

\end{document}


The resulting circuit looks as follows

![Bell state](../images/qiskit_to_latex.png)

## Computation offloading

**LibKet**'s computation offloading model is very similar to that of [CUDA](https://developer.nvidia.com/cuda-downloads) to ease the transition from GPU- to quantum-accelerated computing. The `device.eval(...)` is just one of three ways to run a quantum expression, which we will refer to as **quantum kernel**, on a quantum device.

#### ``LibKet::utils::json device.eval(...)`` 
offloads the quantum computation to the quantum device and returns the evaluated result as JSON object once the quantum computation has completed (exceptions are the QuEST and QX simulators where a reference to the internal state vector is returned).

#### ``LibKet::QJob* device.execute(...)`` 
offloads the quantum computation to the quantum device and returns a ``QJob`` pointer once the quantum computation has completed.

#### ``LibKet::QJob* device.execute_async(...)`` 
offloads the quantum computation to the quantum device and returns a ``QJob`` pointer immediately. \[This cloud-based environment does not allow non-blocking execution so that this method also performs blocking execution.\]

All three methods have the same interface. For Python-based quantum backends this is
```cpp
QJob<QJobType::Python>* execute(std::size_t shots                 = [default from constructor],
                                const std::string& script_init    = "",
                                const std::string& script_before  = "",
                                const std::string& script_after   = "",
                                QStream<QJobType::Python>* stream = NULL)
```
and for backends implemented in C/C++ like QuEST and QX this is
```cpp
QJob<QJobType::CXX>*    execute(std::size_t shots = 0,
                                std::function<void(QDevice_QuEST*)> func_init = NULL,
                                std::function<void(QDevice_QuEST*)> func_before = NULL,
                                std::function<void(QDevice_QuEST*)> func_after = NULL,
                                QStream<QJobType::CXX>* stream = NULL)
```

### Customizing quantum kernel execution

The optional hooks make it possible to inject user-defined code at three different locations of the execution process:

- **init** is performed before any other code of the execution process
- **before** is performed just before sending instructions to the quantum device
- **after** is performed just after receiving the result from the quantum device

Let's inject a simple print statement after the execution

In [ ]:
auto qjob = qiskit.execute_async(1  /* shots         */,
                                 "" /* script_init   */,
                                 "" /* script_before */,
                                 "" /* script_after  */);